In [ ]:
#  Importing Modules
import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedKFold
import scipy.stats as stats
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression, LinearRegression, SGDClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import ElasticNetCV
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier , HistGradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import LinearSVC

## Reading Files for analysis 

In [ ]:
# train = pd.read_csv('Training LUNG Cancer Dataset.csv', low_memory=True)
# melanoma = pd.read_csv('Melanoma Validation Dataset.csv', low_memory=True)
# lancent = pd.read_csv('Lancent_dataset_complete.txt', low_memory=True)
extra_lung = pd.read_csv

In [ ]:

# Functions for filling empty values 
def fill_missing_with_mean(row):
    return row.fillna(row.mean())

# Dataset conversion for validation
lancent_test_standarized = scaler.fit_transform(lancent[rf_cpgs].fillna(0))
lancent_y = lancent['Labels']
melanoma_test_standarized = scaler.fit_transform(melanoma[rf_cpgs].fillna(0))
melanoma_y = melanoma['Labels']

# Functions for calling matrices 
def print_metrics(x, y):
    print("Accuracy:", accuracy_score(x, y))
    print("\nConfusion Matrix:\n", confusion_matrix(x, y))
    print("\nClassification Report:\n", classification_report(x, y))

## OverSampling

In [ ]:
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE
scaler = StandardScaler()


cpg_sites = train.drop(['geo_accession', 'Labels'], axis=1)
labels = train['Labels']


X = scaler.fit_transform(cpg_sites)
Y = labels
#print(X)
print(X.shape)
print(Y.shape)
print ("After OverSampling")
oversampling = SMOTE()
X,Y = oversampling.fit_resample(X,Y)
print(X.shape)
print(y.shape)

(60, 866091)
(60,)
After OverSampling
(90, 866091)
(90,)


## Feature Selection

In [ ]:
RF = RandomForestClassifier()
RF.fit(X,Y)
dfscores = pd.DataFrame(RF.feature_importances_)
dfcolumns = pd.DataFrame(cpg_sites.columns)
fs = pd.concat([dfcolumns, dfscores], axis=1)
fs.columns = ['rfFeature', 'rfScore']
rf_cpgs = fs[fs['rfScore'] > 0].sort_values(by='rfScore', ascending=False)['rfFeature'].tolist()

RandomForestClassifier()

## Testing and training data split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_df, Y, test_size=0.2, random_state=42)

## Model implementation

In [ ]:
model = RandomForestClassifier(**best_params, random_state=0) #new_model
model.fit(X_train.values,y_train)
print("Train : " ,model.score(X_test,y_test))
print("Lancent : ",model.score(lancent_test_standarized,lancent_y))
print("Melanoma : " ,model.score(melanoma_test_standarized,melanoma_y))

## Iterations over several parameters to select model with best accuracy 

In [ ]:
param_grid = {
    'n_estimators': [i for i in range(1,30)],
    'max_depth': [None,1,2,3,4,5,6,7,8,9],
    'min_samples_split': [2,3,4,5,6],
    'min_samples_leaf': [ 1,2,3,4,5],
    'max_features': [None, 'sqrt', 'log2'],
    'n_jobs' : [-1],
}
params_list = []
best_accuracy = 0.0
best_params = None
import itertools
for params in itertools.product(*param_grid.values()):
    param_dict = dict(zip(param_grid.keys(), params))

    rf_classifier = RandomForestClassifier(random_state=0, **param_dict)
    rf_classifier.fit(X_train.values, y_train)

    y_val_pred = rf_classifier.predict(melanoma_test_standarized)
    val_accuracy = accuracy_score(melanoma_y, y_val_pred)
    
    val2_pred = rf_classifier.predict(lancent_test_standarized)
    val2_accuracy = accuracy_score(lancent_y, val2_pred)
    data_dict = {}
    data_dict['Lancent'] = val2_accuracy
    data_dict['Melanoma'] = val_accuracy
    data_dict['Params'] = param_dict
    params_list.append(data_dict)
    pd.DataFrame(params_list).to_csv('RF Lancent Model Params.txt', index=False)
    # val_accuracy = (val_accuracy + val2_accuracy) / 2
    if val_accuracy >= 0.7 and val2_accuracy >= 0.7:
        best_accuracy = val_accuracy
        best_params = param_dict

In [ ]:
# fs[fs['rfScore'] > 0].sort_values(by='rfScore', ascending=False).to_csv('RANDOM FOREST OVERSAMPLED CPGs.csv')
# rf_df = fs[fs['rfScore'] > 0].sort_values(by='rfScore', ascending=False)
# rf_df.sort_values(by='rfFeature')
# res.sort_values(by=['Lancent',"Melanoma"], ascending=False)

## selected model 

In [ ]:
#  Saved MOdel 
import joblib
new_model = joblib.load('Saving Models/RF 70% on both.joblib')
new_model

In [ ]:
model = new_model# RandomForestClassifier(**best_params, random_state=0)
model.fit(X_train.values,y_train)
print("Train : " ,model.score(X_test,y_test))
print("Lancent : ",model.score(lancent_test_standarized,lancent_y))
print("Melanoma : " ,model.score(melanoma_test_standarized,melanoma_y))

Train :  0.8333333333333334
Lancent :  0.7283950617283951
Melanoma :  0.7076923076923077


/home/sbl/.local/lib/python3.8/site-packages/sklearn/base.py:458: UserWarning: X has feature names, but RandomForestClassifier was fitted without feature names
  warnings.warn(
